In [1]:
import numpy as np
import tensorflow as tf
import cv2
import mediapipe as mp

In [2]:
def zero_padding(array):
    threshold = 70
    if array.shape[0] > threshold:
        return array[:threshold, :, :]
    elif array.shape[0] < threshold:
        padding = ((0, threshold - array.shape[0]), (0, 0), (0, 0))
        return np.pad(array, padding, mode='constant', constant_values=0)
    else:
        return array

def extract_landmarks(path: str):

    mp_pose = mp.solutions.pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

    cap = cv2.VideoCapture(path)
    if not cap.isOpened():
        print(f"Error: Could not open video {path}.")
        return
    
    data = []

    with mp_pose as pose:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
        
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = pose.process(rgb_frame)

            if results.pose_landmarks:
                landmarks = [[point.x, point.y, point.z] for point in results.pose_landmarks.landmark]
                data.append(landmarks)

    cap.release()
    np_data = np.array(data)
    return np_data    

In [3]:
x = zero_padding( extract_landmarks( "./sideraise.mp4") )

C:\Users\ASUS\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


In [4]:
x1 = np.expand_dims(  x, axis=0 )
x1.shape

(1, 70, 33, 3)

In [5]:
# x = np.load("./squat.npy")

model = tf.keras.models.load_model("model.h5")

x1 = np.expand_dims(x, axis=0)
x1.shape

maps = {
    0: "Lateral Side Raise",
    1: "Curl",
    2: "Leg Raise",
    3: "Over-Head Press",
    4: "Push-Up",
    5: "Squat",
}

predict = model.predict(x1)

l_p = list(predict[0])
confident = max(l_p)
index = l_p.index(confident)
print(l_p)
print("num class : ", len(l_p))
print("index : ", index)
print("class : ", maps[index])
print("confident : {:.2f} %".format(confident * 100))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/step
[0.0205753, 0.9147661, 6.711563e-06, 0.063961275, 5.651695e-07, 0.00069001416]
num class :  6
index :  1
class :  Curl
confident : 91.48 %
